In [1]:
import os
os.chdir('..')

In [2]:
import torch
import numpy as np
from omegaconf import OmegaConf
from src.scoring import PrecalculationModule

/home/tsa87/anaconda3/envs/tacogfn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [37]:
MODEL_PATH = 'model_weights/model.tar'
HEAD_PATH = 'model_weights/crossdocked_dim_128.pth'

In [38]:
head = torch.load(HEAD_PATH)
score_threshold = head["absolute_score_threshold"]

cfg = OmegaConf.create(head['config'])
focus_threshold = cfg.THRESHOLD.FOCUS
box_threshold = cfg.THRESHOLD.BOX

In [39]:
score_threshold

{'Hydrophobic': 0.12035781145095825,
 'PiStacking_P': 0.0017433513421565294,
 'PiStacking_T': 0.027523474767804146,
 'PiCation_lring': 0.009127387776970863,
 'PiCation_pring': 0.001319622853770852,
 'HBond_ldon': 0.0827205702662468,
 'HBond_pdon': 0.12345150858163834,
 'SaltBridge_lneg': 0.12117615342140198,
 'SaltBridge_pneg': 0.06078973412513733,
 'XBond': 0.0020935165230184793}

In [5]:
# predictor = PrecalculationModule(MODEL_PATH, HEAD_PATH, 'cuda')

In [24]:
from src.pharmaconet.modeling import ModelingModule
module = ModelingModule('model_weights/model.tar', 'cuda',
                        score_threshold=score_threshold)

In [25]:
split_file = torch.load('dataset/pocket_to_ligands.pt')

In [26]:
for rec_id, lig_id in split_file.items():
    rec_path = os.path.join('dataset/crossdock', rec_id + '_rec.pdb')
    lig_path = os.path.join('dataset/crossdocked_pocket10', lig_id)
    
    p_model = module.run(rec_path, ref_ligand_path=lig_path)
    break

In [43]:
p_model.nodes[2].feature.shape

(192,)

In [9]:
# center: [ -5.1225715 -32.597176    6.1419644]

In [23]:
pocket_image2, tokens2 = predictor._PrecalculationModule__parse_protein(rec_path,  np.array([-5.1225715, -32.597176, 6.1419644]))

In [8]:
print(rec_id)
print(len(p_model.nodes))

4xe6_X
64
